In [1]:
#Importaciones basicas
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
# graficos incrustados
%matplotlib inline

#Carga de los CSV
BBVA_data = pd.read_csv('train_clientes.csv', header=0)
print("Data Train: ", BBVA_data.shape)
BBVA_test = pd.read_csv('test_clientes.csv', header=0)
print("Data Test:  ", BBVA_test.shape)

Data Train:  (70000, 53)
Data Test:   (30000, 52)


In [2]:
def completar_data(BBVA):
    if BBVA.isnull().any().any():
        tipos = BBVA.columns.to_series().groupby(BBVA.dtypes).groups
        ctext = tipos[np.dtype('object')]
        columnas = BBVA.columns  # lista de todas las columnas
        cnum = list(set(columnas) - set(ctext))
        #COmpletando los valores nulos de colummnas numericas con la media
        for c in cnum:
            mean = BBVA[c].mean()
            BBVA[c] = BBVA[c].fillna(mean)
        # Completando valores nulos de columnas categóricas con la moda
        for c in ctext:
            mode = BBVA[c].mode()[0]
            BBVA[c] = BBVA[c].fillna(mode)
        print("Clean Data Test: ", BBVA.shape, BBVA.isnull().any().any())

completar_data(BBVA_data)
completar_data(BBVA_test)

Clean Data Test:  (70000, 53) False
Clean Data Test:  (30000, 52) False


In [3]:
def factorizacion_data(datus):
    #Pasando los datos de edad a rango de 10 años
    EDAD2 = pd.cut(datus['EDAD'], range(0, 150, 10))
    datus['EDAD'] = EDAD2
    
    #Factorizando la columna EDAD
    datus['EDAD'], _ = pd.factorize(datus['EDAD'])
    
    #Factorizar las demas columnas de datos categoricos
    tipos = datus.columns.to_series().groupby(datus.dtypes).groups
    ctext = tipos[np.dtype('object')]
    for c in ctext:
        datus[c], _ = pd.factorize(datus[c])
    print(type(datus), datus.shape)
    return datus

BBVA_data = factorizacion_data(BBVA_data)
BBVA_test = factorizacion_data(BBVA_test)

<class 'pandas.core.frame.DataFrame'> (70000, 53)
<class 'pandas.core.frame.DataFrame'> (30000, 52)


In [4]:
print(BBVA_data.isnull().any().any())
print(BBVA_test.isnull().any().any())

False
False


In [5]:
X_train = BBVA_data.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1)
print(type(X_train), X_train.shape)
X_predic = BBVA_test.drop(['ID_CORRELATIVO'], axis=1)
print(type(X_predic), X_predic.shape)
y_train = BBVA_data['ATTRITION'].values
print(type(y_train), y_train.shape)

<class 'pandas.core.frame.DataFrame'> (70000, 51)
<class 'pandas.core.frame.DataFrame'> (30000, 51)
<class 'numpy.ndarray'> (70000,)


In [6]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE

columnas = list(X_train.columns.values)
era = RFE(ExtraTreesClassifier(), 15)  # número de atributos a seleccionar
era = era.fit(X_train.values, y_train)
# imprimir resultados
atrib = era.support_
atributos = [columnas[i] for i in list(atrib.nonzero()[0])]
print(atributos)

['RANG_INGRESO', 'EDAD', 'ANTIGUEDAD', 'RANG_SDO_PASIVO_MENOS0', 'SDO_ACTIVO_MENOS0', 'RANG_NRO_PRODUCTOS_MENOS0', 'NRO_ACCES_CANAL2_MENOS0', 'NRO_ACCES_CANAL3_MENOS0', 'NRO_ACCES_CANAL3_MENOS1', 'NRO_ACCES_CANAL3_MENOS2', 'NRO_ACCES_CANAL3_MENOS3', 'NRO_ACCES_CANAL3_MENOS4', 'NRO_ENTID_SSFF_MENOS0', 'NRO_ENTID_SSFF_MENOS3', 'NRO_ENTID_SSFF_MENOS5']


In [7]:
print(type(X_train), type(X_predic))
xx_train = X_train.loc[:,atributos].values
print(type(X_train), X_train.shape)
xx_predic = X_predic.loc[:,atributos].values
print(type(X_predic), X_predic.shape)

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'> (70000, 51)
<class 'pandas.core.frame.DataFrame'> (30000, 51)


In [8]:
print(type(xx_train), type(y_train))
from imblearn.over_sampling import SMOTE
X_smote, y_smote = SMOTE(kind='svm').fit_sample(xx_train, y_train)

print("X_smote:: ", X_smote.shape)
print("y_smote:: ", y_smote.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
X_smote::  (118355, 10)
y_smote::  (118355,)


In [12]:
from sklearn.preprocessing import StandardScaler
X_smote_std = StandardScaler().fit_transform(X_smote.astype(float))
print("X_smote_std", X_smote_std.shape)
xx_train_std = StandardScaler().fit_transform(xx_train.astype(float))
print("xx_train_std", xx_train_std.shape)
X_train_std = StandardScaler().fit_transform(X_train.values.astype(float))
print("X_train_std", X_train_std.shape)

X_smote_std (118355, 10)
xx_train_std (70000, 10)
X_train_std (70000, 51)


In [13]:
from sklearn import svm
svm.SVC(probability=True)
clf_smote = svm.SVC(probability=True)
clf_20 = svm.SVC(probability=True)
clf_all = svm.SVC(probability=True)
#Construimos el modelo sobre los datos de entrenamiento
clf_smote.fit(X_smote_std,y_smote)
clf_20.fit(xx_train_std,y_train)
clf_all.fit(X_train_std,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
xx_predic.shape

(30000, 10)

In [15]:
X_predic.shape

(30000, 51)

In [16]:
prediccion_smote = clf_smote.predict(xx_predic)
prediccion_proba_smote = clf_smote.predict_proba(xx_predic)
#////////////////////////////////////////////////
prediccion_20 = clf_20.predict(xx_predic)
prediccion_proba_20 = clf_20.predict_proba(xx_predic)
#////////////////////////////////////////////////
prediccion_all = clf_all.predict(X_predic)
prediccion_proba_all = clf_all.predict_proba(X_predic)

In [17]:
print(sum(prediccion_smote), sum(prediccion_20), sum(prediccion_all))

367 31 0


In [18]:
print(len(prediccion_smote), len(prediccion_20), len(prediccion_all))

30000 30000 30000


In [19]:
result_smote = [float("{0:.4f}".format(_)) for _ in list(zip(*prediccion_proba_smote))[1]]
result_20 = [float("{0:.4f}".format(_)) for _ in list(zip(*prediccion_proba_20))[1]]
result_all = [float("{0:.4f}".format(_)) for _ in list(zip(*prediccion_proba_all))[1]]

In [20]:
idd = BBVA_test['ID_CORRELATIVO']
idd.shape

(30000,)

In [21]:
resultados_smote = list(zip(list(idd), result_smote))
resultados_20 = list(zip(list(idd), result_20))
resultados_all = list(zip(list(idd), result_all))

In [22]:
import csv
resultados_smote.insert(0, ('ID_CORRELATIVO', 'ATTRITION'))
print(resultados_smote[:10])
resultados_20.insert(0, ('ID_CORRELATIVO', 'ATTRITION'))
print(resultados_20[:10])
resultados_all.insert(0, ('ID_CORRELATIVO', 'ATTRITION'))
print(resultados_all[:10])

[('ID_CORRELATIVO', 'ATTRITION'), (47411, 0.1879), (39861, 0.046), (38898, 0.1887), (50927, 0.1672), (32969, 0.1692), (89661, 0.3141), (12197, 0.1878), (71520, 0.1879), (59759, 0.0708)]
[('ID_CORRELATIVO', 'ATTRITION'), (47411, 0.3378), (39861, 0.1546), (38898, 0.3209), (50927, 0.2683), (32969, 0.1564), (89661, 0.4732), (12197, 0.3372), (71520, 0.3378), (59759, 0.1275)]
[('ID_CORRELATIVO', 'ATTRITION'), (47411, 0.1823), (39861, 0.1823), (38898, 0.1823), (50927, 0.1823), (32969, 0.1823), (89661, 0.1823), (12197, 0.1823), (71520, 0.1823), (59759, 0.1823)]


In [23]:
with open("imputacion_factorizacion_15_SMOTE_estandarizacion_SVM.csv", 'w') as resultFile:
       wr = csv.writer(resultFile, dialect='excel')
       for row in resultados_smote:
           wr.writerow(row)
with open("imputacion_factorizacion_15_NoBalanceado_estandarizacion_SVM.csv", 'w') as resultFile:
       wr = csv.writer(resultFile, dialect='excel')
       for row in resultados_20:
           wr.writerow(row)
with open("imputacion_factorizacion_allVar_NoBalanceado_estandarizacion_SVM.csv", 'w') as resultFile:
       wr = csv.writer(resultFile, dialect='excel')
       for row in resultados_all:
           wr.writerow(row)

In [26]:
#Probando Regresion Logistica
from sklearn.linear_model import LogisticRegression
classifier_20 = LogisticRegression(random_state=0)
classifier_all = LogisticRegression(random_state=0)

#Construimos el modelo sobre los datos de entrenamiento
classifier_20.fit(xx_train_std,y_train)
classifier_all.fit(X_train_std,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
prediccion_20_log = classifier_20.predict(xx_predic)
prediccion_proba_20_log = classifier_20.predict_proba(xx_predic)
print("prediccion_20_log", sum(prediccion_20_log), prediccion_20_log.shape)
print("prediccion_proba_20_log", sum(prediccion_proba_20_log), prediccion_proba_20_log.shape)
#////////////////////////////////////////////////
prediccion_all_log = classifier_all.predict(X_predic)
prediccion_proba_all_log = classifier_all.predict_proba(X_predic)
print("prediccion_all_log", sum(prediccion_all_log), prediccion_all_log.shape)
print("prediccion_proba_all_log", sum(prediccion_proba_all_log), prediccion_proba_all_log.shape)

prediccion_20_log 265 (30000,)
prediccion_proba_20_log [ 29042.37846512    957.62153489] (30000, 2)
prediccion_all_log 2426 (30000,)
prediccion_proba_all_log [ 26721.24817004   3278.75182996] (30000, 2)


/home/villacorta/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [29]:
result_20_log = [float("{0:.4f}".format(_)) for _ in list(zip(*prediccion_proba_20_log))[1]]
result_all_log = [float("{0:.4f}".format(_)) for _ in list(zip(*prediccion_proba_all_log))[1]]

resultados_20_log = list(zip(list(idd), result_20_log))
resultados_all_log = list(zip(list(idd), result_all_log))

In [30]:
import csv
resultados_20_log.insert(0, ('ID_CORRELATIVO', 'ATTRITION'))
print(resultados_20_log[:10])
resultados_all_log.insert(0, ('ID_CORRELATIVO', 'ATTRITION'))
print(resultados_all_log[:10])

[('ID_CORRELATIVO', 'ATTRITION'), (47411, 0.0002), (39861, 0.0564), (38898, 0.0004), (50927, 0.0009), (32969, 0.0415), (89661, 0.0044), (12197, 0.0031), (71520, 0.0), (59759, 0.0168)]
[('ID_CORRELATIVO', 'ATTRITION'), (47411, 0.004), (39861, 0.0888), (38898, 0.0032), (50927, 0.0027), (32969, 0.0527), (89661, 0.0064), (12197, 0.005), (71520, 1.0), (59759, 0.0167)]


In [31]:
with open("imputacion_factorizacion_20_NoBalanceado_RegresionLogistica.csv", 'w') as resultFile:
       wr = csv.writer(resultFile, dialect='excel')
       for row in resultados_20_log:
           wr.writerow(row)
with open("imputacion_factorizacion_allVar_NoBalanceado_RegresionLogistica.csv", 'w') as resultFile:
       wr = csv.writer(resultFile, dialect='excel')
       for row in resultados_all_log:
           wr.writerow(row)